In [54]:
import numpy as np
import glob
from scipy.integrate import simps

from gwbench import snr
from gwbench import waveform as wfc
from gwbench import injections
from gwbench import network
from gwbench import basic_relations as br

from pycbc import conversions as conv
from  pycbc.filter.matchedfilter import optimized_match

from astropy.cosmology import Planck18, z_at_value

import gwbench_network_funcs as gwnet

from pycbc.types import FrequencySeries
from pycbc.filter import match
from pycbc.filter import matchedfilter

from pycbc import types, fft, waveform

In [2]:
import matplotlib.pyplot as plt

import matplotlib as mpl

plt.rcParams.update({
    "text.usetex": True,
    "font.family": "serif",
    "font.size": 18
})


%config InlineBackend.figure_format='retina' # very useful command for high-res images

In [12]:
# takes ~ 2 min
files = glob.glob1('powerlaw_smooth_hybrid_3G_production/', 'hybr_0.0_*')
print(files)

df = pd.DataFrame()
for fi in files[0:2]:
    df_temp = pd.read_csv('powerlaw_smooth_hybrid_3G_production/'+fi)
    df = pd.concat([df,df_temp])
# drop the zero rows that resulted from NoneType Networks
df = df.loc[~(df==0).all(axis=1)]

print(len(df[(df["hybr"]==0.0)]))

['hybr_0.0_0_20000.csv', 'hybr_0.0_20000_40000.csv', 'hybr_0.0_60000_80000.csv', 'hybr_0.0_100000_100259.csv', 'hybr_0.0_99741_100000.csv', 'hybr_0.0_40000_60000.csv', 'hybr_0.0_80000_100000.csv']
40000


In [14]:
print("Median SNR:", np.median(df[(df["hybr"]==0.0)]["snr"]))

print(">0:", np.sum((df["hybr"]==0.0) & (df["snr"]>0)))
print(">10:", np.sum((df["hybr"]==0.0) & (df["snr"]>10)))
print(">50:", np.sum((df["hybr"]==0.0) & (df["snr"]>50)))
print(">100:", np.sum((df["hybr"]==0.0) & (df["snr"]>100)))
print(">200:", np.sum((df["hybr"]==0.0) & (df["snr"]>200)))
print(">300:", np.sum((df["hybr"]==0.0) & (df["snr"]>300)))

Median SNR: 22.209034499015274
>0: 40000
>10: 35498
>50: 5036
>100: 1202
>200: 357
>300: 206


# Compute Ratio of LSA likelihood to Exact Likelihood (from Vallisneri 2017)

In [67]:
def scalar_product_integrand(hf, gf, psd):
    temp = np.multiply(hf, np.conj(gf))
    return 2 * np.divide(temp + np.conj(temp), psd)

def scalar_product_freq_array(hf, gf, psd, freqs):
    return np.real(simps(scalar_product_integrand(hf, gf, psd), freqs))

In [15]:
input_file = 'smooth_powerlaw_pop.npz'
with np.load(input_file, mmap_mode='r') as binaries:
    Mcs = np.array(binaries['Mcs'])
    etas = np.array(binaries['etas'])
    chi1z = np.array(binaries['chi1z'])
    chi2z = np.array(binaries['chi2z'])
    DLs = np.array(binaries['DLs'])
    iotas = np.array(binaries['iotas'])
    ras = np.array(binaries['ras'])
    decs = np.array(binaries['decs'])
    psis = np.array(binaries['psis'])

m1s = conv.mass1_from_mchirp_eta(Mcs, etas)
mtots = conv.mtotal_from_mchirp_eta(Mcs, etas)



In [52]:
i = 54837

inj_params = {
    'Mc':    Mcs[i],
    'eta':   etas[i],
    'chi1x': 0.,
    'chi2x': 0.,
    'chi1y': 0.,
    'chi2y': 0.,
    'chi1z': chi1z[i],
    'chi2z': chi2z[i],
    'DL':    DLs[i],
    'tc':    0,
    'phic':  0,
    'iota':  iotas[i],
    'ra':    ras[i],
    'dec':   decs[i],
    'psi':   psis[i],
    'gmst0': 0,
    'hybr': 0.0
} 

net_key = '3G'

d_f = 2**-4
f_low = 5.0
mtotal = conv.mtotal_from_mchirp_eta(inj_params['Mc'], inj_params['eta'])
f_high = np.round(4*br.f_isco_Msolar(mtotal))

net_ap = gwnet.get_network_response(inj_params=inj_params, f_max=f_high, approximant="IMRPhenomD", network_key=net_key, calc_detector_responses=True)

2023-10-26 14:49:41,916 - Network - INFO : Polarizations calculated.
2023-10-26 14:49:41,917 - Network - INFO : Calculate numeric derivatives of polarizations.
2023-10-26 14:49:41,920 - Network - INFO : Polarizations calculated.
2023-10-26 14:49:42,591 - Network - INFO : Numeric derivatives of polarizations calculated.
2023-10-26 14:49:42,640 - Network - INFO : PSDs loaded.
2023-10-26 14:49:42,642 - Network - INFO : Antenna patterns and LPFs loaded.
2023-10-26 14:49:42,653 - Network - INFO : Detector responses calculated.
2023-10-26 14:49:42,653 - Network - INFO : Calculate numeric derivatives of detector responses.
2023-10-26 14:49:42,654 - Network - INFO :    CE-40_C
2023-10-26 14:49:43,091 - Network - INFO :    CE-20_S
2023-10-26 14:49:43,520 - Network - INFO :    ET_ET1
2023-10-26 14:49:43,965 - Network - INFO :    ET_ET2
2023-10-26 14:49:44,402 - Network - INFO :    ET_ET3
2023-10-26 14:49:44,834 - Network - INFO : Numeric derivatives of detector responses calculated.
2023-10-26 1

In [66]:
inj_params_1_sig = net_ap.inj_params.copy()

for param in net_ap.deriv_variables:
    inj_params_1_sig[param] = net_ap.inj_params[param] + net_ap.errs[param]

print(inj_params_1_sig)
net_1_sig = gwnet.get_network_response(inj_params=inj_params_1_sig, f_max=f_high, approximant="IMRPhenomD", network_key=net_key, calc_detector_responses=True)

2023-10-26 14:59:10,417 - Network - INFO : Polarizations calculated.


2023-10-26 14:59:10,426 - Network - INFO : Calculate numeric derivatives of polarizations.
2023-10-26 14:59:10,435 - Network - INFO : Polarizations calculated.


{'Mc': 39.100505778702114, 'eta': 0.24901617667854037, 'chi1x': 0.0, 'chi2x': 0.0, 'chi1y': 0.0, 'chi2y': 0.0, 'chi1z': 0.15075946210840996, 'chi2z': 0.2207446966565786, 'DL': 1987.1097455046167, 'tc': 0.0010416506556794047, 'phic': 0.727260947227478, 'iota': 0.5246762329548236, 'ra': 0.16787946892691702, 'dec': -0.35175208133766867, 'psi': 4.775045584023324, 'gmst0': 0, 'hybr': 0.0}


2023-10-26 14:59:11,268 - Network - INFO : Numeric derivatives of polarizations calculated.
2023-10-26 14:59:11,340 - Network - INFO : PSDs loaded.
2023-10-26 14:59:11,342 - Network - INFO : Antenna patterns and LPFs loaded.
2023-10-26 14:59:11,354 - Network - INFO : Detector responses calculated.
2023-10-26 14:59:11,355 - Network - INFO : Calculate numeric derivatives of detector responses.
2023-10-26 14:59:11,356 - Network - INFO :    CE-40_C
2023-10-26 14:59:12,019 - Network - INFO :    CE-20_S
2023-10-26 14:59:12,673 - Network - INFO :    ET_ET1
2023-10-26 14:59:13,317 - Network - INFO :    ET_ET2
2023-10-26 14:59:13,948 - Network - INFO :    ET_ET3
2023-10-26 14:59:14,598 - Network - INFO : Numeric derivatives of detector responses calculated.
2023-10-26 14:59:14,601 - Network - INFO : SNRs calculated.
2023-10-26 14:59:14,601 - Network - INFO : Calculate errors (Fisher & cov matrices).
2023-10-26 14:59:14,602 - Network - INFO :    CE-40_C
2023-10-26 14:59:14,662 - Network - INFO :

In [82]:
net_1_sig.detectors[0].del_hf

{'del_Mc_hf': array([-6.35980465e-25-1.33736297e-24j, -1.60165105e-23-1.63697621e-23j,
         5.70620513e-24-4.39737654e-23j, ...,
         1.34903135e-23-1.37859792e-23j,  1.34319921e-23-1.38228898e-23j,
         1.33735379e-23-1.38596228e-23j]),
 'del_eta_hf': array([-5.54264123e-24-1.16552685e-23j,  1.66735134e-22+1.32703817e-22j,
        -2.73667931e-23+4.14946877e-22j, ...,
        -4.02451962e-22+4.27933414e-22j, -4.00681613e-22+4.29134813e-22j,
        -3.98904757e-22+4.30329194e-22j]),
 'del_chi1z_hf': array([-3.77437579e-25-7.93689536e-25j,  2.30175569e-23+1.95241528e-23j,
        -4.80936930e-24+5.87144698e-23j, ...,
        -4.07841244e-23+4.22496145e-23j, -4.06064455e-23+4.23632450e-23j,
        -4.04281497e-23+4.24761333e-23j]),
 'del_chi2z_hf': array([-3.23168511e-25-6.79570557e-25j,  1.95340533e-23+1.65604961e-23j,
        -4.07355658e-24+4.98141632e-23j, ...,
        -3.08290590e-23+3.19564844e-23j, -3.06935143e-23+3.20412182e-23j,
        -3.05575255e-23+3.21253970e-

In [84]:
log_r_j = np.zeros(len(net_ap.deriv_variables))

d = 0

for j, param_j in enumerate(net_ap.deriv_variables):
    theta_1_sig_j = net_1_sig.inj_params[param_j]

    h_j = net_ap.detectors[d].hf
    h_1_sig = net_1_sig.detectors[d].hf
    delta_h_1_sig_j = h_1_sig - h_j
    
    a = (theta_1_sig_j*net_1_sig.detectors[d].del_hf['del_'+param_j+'_hf']) - delta_h_1_sig_j

    for k, param_k in enumerate(net_ap.deriv_variables):
        theta_1_sig_k = net_1_sig.inj_params[param_k]

        h_k = net_ap.detectors[d].hf
        h_1_sig = net_1_sig.detectors[d].hf
        delta_h_1_sig_k = h_1_sig - h_k

        b = (theta_1_sig_k * net_1_sig.detectors[d].del_hf['del_'+param_k+'_hf']) - delta_h_1_sig_k

        log_r_j[j] += scalar_product_freq_array(a, b, psd=net_ap.detectors[d].psd, freqs=net_ap.detectors[d].f)/2


In [85]:
log_r_j

array([ 3.72998601e+10, -4.54045695e+09, -3.38636542e+08, -4.11668666e+08,
        8.81480671e+06, -9.17921864e+06,  5.61149955e+07,  6.44772903e+06,
       -4.39840514e+07,  8.82142102e+07, -6.38071621e+08])